In [1]:
try:
    import elasticsearch
    from elasticsearch import Elasticsearch
    import pandas as pd
    import json
    from ast import literal_eval
    from tqdm import tqdm
    import datetime
    import os
    import sys
    import numpy as np
    from elasticsearch import helpers
    print('All libraries loaded successfully')
    
except Exception as e:
    print(e)

All libraries loaded successfully


In [2]:
def connect_elasticsearch():
    es = None
    es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
    if es.ping():
        print('Connected ')
    else:
        print('Could not connect!')
    return es
es = connect_elasticsearch()

Connected 


In [3]:
df = pd.read_csv('netflix_titles.csv')
# drop all the null values
df = df.dropna()
# convert a dataframe to dictionary
df_dict = df.to_dict('records')

def generator(index_name, df):
    for item, line in enumerate(df):
        yield{
            '_index': index_name,
            '_type': '_doc',
            '_id': line.get('show_id', None),
            '_source': {
                'title': line.get('title', ''),
                'director': line.get('director', ''),
                'cast': line.get('cast', ''),
                'country': line.get('country', ''),
                'release_year': line.get('release_year', '')
            }
        }
    raise StopIteration
    
settings = {
    'settings':{
        'number_of_shards':1,
        'number_of_replicas':0
    },
    'mappings':{
        'properties':{
            'director':{
                'type': 'text'
                
            },
            'cast':{
                'type': 'text'
            }
            
        }
    }
}
    
my = es.indices.create(index = 'netflix_titles_elk', 
                       ignore = [404, 400], 
                       body = settings)

In [4]:
index_name = 'netflix_titles_elk'
try:
    res = helpers.bulk(client = es, 
                       actions = generator(index_name, df_dict))
    print('completed')
except Exception as e:
    print(e)
    pass

C:\Users\Arjun Janamatti\Anaconda3\envs\tf_nptel\lib\site-packages\elasticsearch\connection\base.py:193: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


completed


In [5]:
# query


myquey = {
   "_source": [],
   "size": 10,
   "query": {
      "bool": {
         "must": [],
         "filter": [
            {
               "exists": {
                  "field": "director"
               }
            }
         ],
         "should": [
            {
               "match": {
                  "director": "Vikram"
               }
            }
         ],
         "must_not": []
      }
   }
}



In [8]:
res = es.search(
  index = 'netflix_titles_elk',
  scroll = '2m',
  size = 10,
  body = myquey)

In [9]:
res

{'_scroll_id': 'DXF1ZXJ5QW5kRmV0Y2gBAAAAAAAABx0WZnNDUkR3ZDBTSnVVdFN1bXYtVmN0QQ==',
 'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 4804, 'relation': 'eq'},
  'max_score': 7.263006,
  'hits': [{'_index': 'netflix_titles_elk',
    '_type': '_doc',
    '_id': 's10',
    '_score': 7.263006,
    '_source': {'title': '1920',
     'director': 'Vikram Bhatt',
     'cast': 'Rajneesh Duggal, Adah Sharma, Indraneil Sengupta, Anjori Alagh, Rajendranath Zutshi, Vipin Sharma, Amin Hajee, Shri Vallabh Vyas',
     'country': 'India',
     'release_year': 2008.0}},
   {'_index': 'netflix_titles_elk',
    '_type': '_doc',
    '_id': 's756',
    '_score': 7.263006,
    '_source': {'title': 'Barry',
     'director': 'Vikram Gandhi',
     'cast': 'Devon Terrell, Anya Taylor-Joy, Jason Mitchell, Ashley Judd, Jenna Elfman, Ellar Coltrane, Avi Nash, Linus Roache, John Benjamin Hickey',
     'country': 'United States',
     're